# A pipeline for recording, processing & successfully merging e-phys, IMU and tracking data (v3.0.0).
### author: github/bartulem


#### **[Step 0]** Considerations before, during and after conducting the experiments.

1. Set up [SpikeGLX](https://billkarsh.github.io/SpikeGLX/) to accommodate your specific recording configuration.
2. Turn on and calibrate the IMU before every session (system calibration does not need to be 3, but others do before the rat is plugged on; time readings should not have duplicates either).
3. Open Motive.
- Check whether the system is calibrated (continuous calibration should be on).
- If necessary, for each camera change strobe light settings to continuous light.
- Check that the rigid bodies for the head & arena LEDs exist (this enables on-line automatic labeling).
- Check whether the acquisition directory is the correct one.
- Check whether camera 1 is recording in MJPG/greyscale mode.
4. Put three circular markers on the back of the animal and plug it on.
5. Conduct the recording.
- In the NPX, tracking and IMU acquisition programs, you should see the microcontroller-generated random LED pulses.
- Start acquiring the IPI sync data.
- Start recording in SpikeGLX.
- Start acquiring data on the IMU.
- Start recording in Motive.
- Keep it going for some time (e.g. 20-25 min).
- Stop recording in Motive.
- Stop acquiring data on the IMU.
- Stop recording in SpikeGLX.
- Stop acquiring the IPI sync data.
6. It's good practice to label the back points immediately in Motive (the head & LEDs should be labeled already, if step 3d was implemented) and export the data in a .csv file.

#### **[Step 1]** Concatenate Neuropixel recording sessions. This step is *optional*, you can skip it if you are interested in only one session.

As a necessary input, you determine:
1. the directories where the NPX .bin files are (all associated files should be in the same directory; also, make sure they're named in a way that will order them properly)
2. the desired paths to the future merged files
3. the desired paths to the future .pkl files

As elective inputs, note that you have the option to set:
1. file_type (concatenate lf or ap files; defaults to ap)
2. cmd_prompt (whether to do the merging through the terminal/cmd prompt; defaults to True)
3. nchan (the number of channels on the probe; defaults to 385)
4. npx_sampling_rate (sampling rate of the NPX system; defaults to 3e4)

Thus, multiple unrelated data streams can be processed sequentially.

The outputs of the code are as follows:
1. new_file_name (the newly created concatenated file; saved as .bin file)
2. pkl_len (information about change-points of the concatenated sessions; saved as .pkl file)

In [ ]:
from kisn_pylab import concatenate

file_directories = [r'A:\store\Bartul\neuropixel\26525_kavorka_RH\190620_distal\spikes_imec0']
new_file_names = [r'A:\store\Bartul\neuropixel\26525_kavorka_RH\190620_distal\spikes_imec0\190620_distal_all_g0_t0.imec0.ap.bin']
pkl_lengths = [r'A:\store\Bartul\neuropixel\26525_kavorka_RH\190620_distal\spikes_imec0\190620_distal_all_g0_t0.imec0.ap.pkl']

In [ ]:
for file_dir, new_file_name, pkl_len in zip(file_directories, new_file_names, pkl_lengths):
    concatClass = concatenate.Concat(file_dir, new_file_name, pkl_len)
    concatClass.concat_npx()

#### **[Step 2]** Run Kilosort2 through Python.

This step assumes you are happy with *everything* in the config file. If you need to modify anything, either the code needs to change or you complete this step in Matlab.

If this doesn't bother you, then you should do the following:
1. Download/clone [Kilosort2](https://github.com/MouseLand/Kilosort2) and set up the config, master and CUDA files accordingly.
2. Install [matlab engine](https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html) (only tried this as an admin, but might work otherwise as well).
3. Kilosort2 runs on all the .bin files in the given directory below. Make sure that this is what you want.
4. Don't use my Kilosort2 directory, but rather your own (created in step 0).
5. Modify the "master" file such that it takes .bin file directory as input.
6. Set the file and Kilosort2 directories, and run the cell below.

!NB: While Kilosort2 is running, it's a good opportunity to label the tracking data if you haven't done so already!

As necessary inputs, note that you have to set:
1. file_dir (the absolute path to the directories the binary files are in)
2. kilosort2_dir (the absolute path to the directory the Kilosort2 code is in)

Thus, multiple unrelated data streams can be processed sequentially.

In [ ]:
from kisn_pylab import kilosort

file_dirs = [r'A:\store\Bartul\neuropixel\26525_kavorka_RH\190620_intermediate\spikes_imec0']
kilosort2_dir = r'A:\group\bartulm\Kilosort2-master'

In [ ]:
for file_dir in file_dirs:
    kilosort.run_kilosort(file_dir, kilosort2_dir)

#### **[Step 3]** Arbitrate what is noise and what are clusters in Phy.
1. Install Phy: [Phy v2.0](https://github.com/cortex-lab/phy)
2. Navigate to the directory where Kilosort2 results were saved, open powershell and type "cmd", followed by "activate phy2", followed by "phy template-gui params.py".
3. Complete the manual curation ([Phy tutorial](https://phy.readthedocs.io/en/latest/)) and save your work.

#### **[Step 4]** Compute cluster quality measures for 'good' and 'MUA' clusters.

This step assumes manual curation was completed (NB: noise was labeled as noise), in order to compute quality measures for non-noise clusters.

As a necessary input, you determine:
1. the directories where the Kilosort2 output files are

As elective inputs, note that you have the option to set:
1. nchan (the number of channels on the probe; defaults to 385)
2. npx_sampling_rate (the sampling rate of the NPX system; defaults to 3e4)
3. num_channels_to_compare (the number of channels to look at around peak channel, must be odd; defaults to 13)
4. max_spikes_for_cluster (the maximum number of spikes to analyze; defaults to 1000)
5. perform_waveforms (yey or ney on the waveform metrics; defaults to True)
6. perform_isiv (yey or ney on the isi_violations computation; defaults to True)
7. perform_mahalanobis (yey or ney on the mahalanobis_metrics; defaults to True)
8. perform_nnm (yey or ney on the nearest_neighbors_metrics; defaults to True)
9. perform_lda (yey or ney on the LDA metric; defaults to True)
10. re_categorize (yey or ney on re-categorizing clusters based on contamination features; defaults to True)
11. min_spikes (the minimum number of spikes a cluster should have to be saved; defaults to 100)
12. max_good_isi (the maximum acceptable FP ISI metric in order for the cluster to be considered 'good'; defaults to .02)
13. max_good_contam (the maximum acceptable ContamPct in order for the cluster to be considered 'good'; defaults to 10)

Thus, multiple unrelated data streams can be processed sequentially.

The outputs of the code are as follows:
1. cluster_quality_measures (a dictionary with cluster quality measures for each non-noise cluster; saved as .json file)
2. cluster_group (a DataFrame with information about cluster type; overwritten .tsv file if re_categorize was chosen)
3. cluster_info (a DataFrame with different information about all clusters; overwritten .tsv file if re_categorize was chosen)

In [ ]:
from kisn_pylab import clusters2categories

kilosort_output_dirs = [r'A:\store\Bartul\neuropixel\26472_roy_LH\270520_intermediate\spikes_imec0']

In [ ]:
for kilosort_output_dir in kilosort_output_dirs:
    qualityClass = clusters2categories.ClusterQuality(kilosort_output_dir=kilosort_output_dir)
    qualityClass.compute_quality_measures()

#### **[Step 5]** Read in the sync events (make sure the PC has enough memory to run this, say 64Gb RAM) and put them in separate .txt files.

If you haven't done so already, label the tracked rigid bodies and marker sets in Motive (read the tutorial if you need) and export the data:
1. File > Export Tracking Data.
2. The following options need to be OFF: (1) Unlabeled markers, (2) Rigid Bodies, (3) Rigid Body markers, (4) Bones, (5) Bone markers.
3. Click "Export" and you should have created a .csv file (it may take ~1 minute, depending on the length of the recording).

As necessary inputs, you determine:
1. the list with the files whose sync events you'd like to read (in practice this would be the imec0 and imec1 files for a given recording session)
2. the absolute path of the future sync .pkl file

As elective inputs, note that you have the option to set:
1. nchan (the number of channels on the probe; defaults to 385)
2. sync_chan (the specific sync port channel on the probe; defaults to 385)
3. track_file (the absolute path to the tracking file for that session; defaults to 0)
4. imu_file (the absolute path to the IMU file for that session; defaults to 0)
5. sync_ipi_file (the absolute path to the sync IPI data .txt file; defaults to 0)
6. sync_led_duration (the duration of ON time for sync LEDs; defaults to 250 (ms))
7. sync_led_error (the possible sample error in the duration of ON time for sync LEDs; defaults to 50 (ms))
8. ground_probe (in a dual probe setting, the probe the other is synced to - if you only have imec1, this needs to be set to 1; defaults to 0)
9. frame_rate (the tracking camera frame rate for that session; defaults to 120)
10. npx_sampling_rate (the sampling rate of the NPX system; defaults to 3e4)
11. sync_sequence (the length of the sequence the LED events should be matched across data streams; defaults to 10)
12. sample_error (the time the presumed IMEC/IMU LEDs could be allowed to err around; defaults to 30 (ms))
13. which_imu_time (the IMU time to be used in the analyses, loop.starttime (0) or sample.time (1); defaults to 1)

Therefore, you proceed one recording session at a time.

The outputs of the code are as follows:
1. track_file (the corrected and LED-shortened tracing data; saved as a separate .csv file if track_file was given).
2. imu_file (the IMU data; saved as .pkl file if IMU .txt data file was given).
3. sync_df (the sync data; saved as .pkl file if either of the two above were given).

In [ ]:
from kisn_pylab import reader

npx_files = [r'D:\SGL_DATA\26504_jacopo_150620\s4_2025_light_intermediate_g0\s4_2025_light_intermediate_g0_imec0\s4_2025_light_intermediate_g0_t0.imec0.ap.bin']
sync_df = r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\sync_df_150620_s4_intermediate.pkl'
track_file = r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\tracking\Take 2020-06-15 08.24.44 PM (1).csv'
imu_file = r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\IMU\CoolTerm Capture 2020-06-15 20-24-41.txt'
sync_ipi_file = r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\IMU\CoolTerm Capture 2020-06-15 20-24-41.pkl'

In [ ]:
readClass = reader.EventReader(npx_files, sync_df)
readClass.read_se(track_file=track_file, imu_file=imu_file, sync_ipi_file=sync_ipi_file)

#### **[Step 6]** Load the sync data from the .pkl file(s) and analyze how well the tracking/IMU data are synced with NPX data.

Before running this step, make sure you have [plotly](https://plotly.com/python/getting-started/?utm_source=mailchimp-jan-2015&utm_medium=email&utm_campaign=generalemail-jan2015&utm_term=bubble-chart) installed.

As a necessary input, you determine:
1. the sync_df .pkl files

As elective inputs, note that you have the option to set:
1. npx_sampling_rate (sampling rate of the NPX system; defaults to 3e4)
2. to_plot (plot or not to plot y_test and y_test_prediction statistics; defaults to False)
3. ground_probe (in a dual probe setting, the probe the other is synced to; defaults to 0)
4. imu_files (the list of absolute paths to imu_pkl files that contain the raw IMU data; defaults to 0)
5. which_imu_time (the IMU time to be used in the analyses, loop.starttime (0) or sample.time (1); defaults to 1)

Thus, multiple files can be processed in sequence.

The imu_files should be ordered such that the first sync file corresponds to the IMU file of the same session, and so forth.

The output of the code is as follows:
1. sync_df (the sync data; overwritten .pkl file)

In [ ]:
from kisn_pylab import synchronize

sync_pkls = [r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\sync_df_150620_s4_intermediate.pkl']
imu_files = [r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\IMU\CoolTerm Capture 2020-06-15 20-24-41.pkl']
to_plot = 1

In [ ]:
syncClass = synchronize.Sync(sync_pkls)
syncClass.estimate_sync_quality(to_plot=to_plot, imu_files=imu_files)

#### **[Step 7]** Split clusters back into individual sessions and get spike times. This step should be completed irrespective of the number of sorted sessions.

As necessary inputs, you determine:
1. the_dirs (a list of directories where Kilosort2 results are stored for each recording probe)
2. sync_pkls (paths to as many sync .pkl files as there are recording sessions)

As elective inputs, note that you have the option to set:
1. nchan (the number of channels on the probe; defaults to 385)
2. one_session (whether you have only one session; defaults to True)
3. min_spikes (the minimum number of spikes in one session to consider the cluster worthy of saving; defaults to 100)
4. npx_sampling_rate (sampling rate of the NPX system; defaults to 3e4)
5. ground_probe (in a dual probe setting, the probe the other is synced to; defaults to 0)
6. to_plot (plot or not to plot y_test and y_test_prediction statistics; defaults to False)
7. pkl_lengths (.pkl files that have information about where concatenated files were stitched together; defaults to 0)
8. print_details (whether or not to print details about spikes in every individual cluster; defaults to 0)
9. important_cluster_groups (the list of relevant cluster groups you want to analyze, should be 'good' and 'mua'; defaults to list('good'))
10. eliminate_duplicates (whether or not to eliminate duplicate spikes; defaults to 1)
11. min_isi (threshold for duplicate spikes in seconds; defaults to 0)
12. switch_clock (convert each sample spike time to time as measured by the IPI generator; defaults to False)

!NB: make sure each directory has the imec ID in the name!

Therefore, you proceed one recording day at a time.

The outputs of the code are as follows:
1. spike times (arrays that contain spike times (in seconds); saved as .mat files in a separate directory)
2. cluster_groups_information (information about which cluster belongs to 'good' or 'MUA' categories; saved as .json file)

In [ ]:
from kisn_pylab import spikes2sessions

the_dirs = [r'A:\store\Bartul\neuropixel\26471_johnjohn_RH\210520\spikes_imec0']
sync_pkls = [r'A:\store\Bartul\neuropixel\sync_df_215020_s1_distal.pkl',
             r'A:\store\Bartul\neuropixel\sync_df_215020_s2_distal.pkl',
             r'A:\store\Bartul\neuropixel\sync_df_215020_s3_distal.pkl',
             r'A:\store\Bartul\neuropixel\sync_df_215020_s4_distal.pkl']
pkl_lengths = [r'A:\store\Bartul\neuropixel\26471_johnjohn_RH\210520\spikes_imec0\210520_distal_all_g0_t0.imec0.ap.pkl']
one_session = False
important_cluster_groups = ['good', 'mua']
min_spikes = 100

In [ ]:
sstClass = spikes2sessions.ExtractSpikes(the_dirs, sync_pkls)
sstClass.split_clusters(one_session=one_session, min_spikes=min_spikes, pkl_lengths=pkl_lengths, important_cluster_groups=important_cluster_groups)

#### **[Step 8]** Create .pkl file for GUI.

As necessary inputs, you determine:
1. the absolute paths where the .csv (tracking) files are (tracking files with the appendage "final" should be used!)
2. the absolute paths where the .pkl (sync event) files are

As elective inputs, note that you have the option to set:
1. frame_rate (you set it manually, otherwise it's read from the sync .pkl file)
2. npx_sampling_rate (sampling rate of the NPX system; defaults to 3e4)
3. ground_probe (in a dual probe setting, the probe the other is synced to; defaults to 0)
4. session_timestamps (whether to take session timestamps (True) for start/stop recording or tracking (False); defaults to True)

Thus, multiple files can be processed in sequence. Every GUI .pkl file is saved to the same directory as the tracking .csv file.

!NB: The rat-cam is meant to be used for the raw tracking video, which should be exported to match the sequence from the start of first to the start of the last LED event!

The output of the code is as follows:
1. final (dictionary with the tracking data and other information; saved as .pkl file)

In [ ]:
from kisn_pylab import motive2GUI

the_csvs = [r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\tracking\Take 2020-06-15 08.24.44 PM (1)_final.csv']
sync_pkls = [r'A:\store\Bartul\neuropixel\26504_jacopo_LH\150620_intermediate\26504_jacopo_150620_2025_intermediate_s4_light\sync_df_150620_s4_intermediate.pkl']

In [ ]:
for the_csv, sync_pkl in zip(the_csvs, sync_pkls):
    mtgClass = motive2GUI.Transformer(the_csv, sync_pkl)
    mtgClass.csv_to_pkl()

#### **[Step 9]** Finish processing the tracking data and create .mat files in the GUI.

1. create the head in the GUI (trackedpointdata_V3_5_LEDs.py version)
2. load the spiking .mat files
3. export everything as a .mat file

#### **[Step 10]** Re-head .mat files according to a template file.

As necessary inputs, you determine:
1. the absolute paths where the template .mat (tracking + spikes) file is
2. the absolute paths where the other .mat files are (!NB: the rigid body placement should be same as in the template file!)

Every processed .mat file is saved to the same directory as the original .mat file.

In [ ]:
from kisn_pylab import rehead

template_file = r'C:\Users\bartulm\Downloads\test_bens_code\bruno_060520_s1_light.mat'
other_files = [r'C:\Users\bartulm\Downloads\test_bens_code\bruno_060520_s2_light.mat']

In [ ]:
reheaderClass = rehead.ReHead(template_file, other_files)
reheaderClass.conduct_transformations()